# Effect of Land Cover on Pollution in Chesapeake Watershed
## Hack the Bay 2020

### Project by: C Ellison, Megan Maloney, and Joe Barrow

Our product uses a model trained on the land cover of upstream area of stations to allow a user to predict pollution given land cover parameters of their choosing. We also create a table with the latitude/longitude of each CBP station and the percent of each landcover type of all HUC12 encompassing upstream bodies of water.

Data:
-NCLD: https://www.mrlc.gov/

-Water Quality (CBP stations): https://github.com/Hack-the-Bay/hack-the-bay

Background information from: https://www.chesapeakebay.net/what/programs/monitoring

Template for some of the code: https://github.com/moezilla-ml/DatabricksHackathon

In [1]:
#load modules 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
import descartes 
import geopandas 
import seaborn as sns

In [3]:
#combine streamflow land cover with locations
file_location = 'stationLocations.csv'
trueLocations = pd.read_csv(file_location, header = None)
trueLocations.columns = ['Latitude','Longitude','numEntries','ID']
file_location = 'StreamFlow_Cover_full.csv'
streamflow = pd.read_csv(file_location)

true_locations_streamflow =  pd.merge(trueLocations, streamflow, how = 'left', left_on = 'ID', right_on = 'FullID')
true_locations_streamflow

,Latitude,Longitude,numEntries,ID,FID,Field1,Field2,Field3,FullID,FullID_Str,...,PctVALUE_42,PctVALUE_43,PctVALUE_45,PctVALUE_46,PctVALUE_52,PctVALUE_71,PctVALUE_81,PctVALUE_82,PctVALUE_90,PctVALUE_95
0,36.740040,-76.584300,60,0,0.0,36.740040,-76.584300,60.0,0.0,0.0,...,0.115285,0.077527,0.016062,0.006030,0.008989,0.001771,0.002487,0.190740,0.335356,0.020549
1,36.740257,-76.579794,504,1,1.0,36.740257,-76.579794,504.0,1.0,1.0,...,0.115285,0.077527,0.016062,0.006030,0.008989,0.001771,0.002487,0.190740,0.335356,0.020549
2,36.740970,-76.577900,70,2,2.0,36.740970,-76.577900,70.0,2.0,2.0,...,0.115285,0.077527,0.016062,0.006030,0.008989,0.001771,0.002487,0.190740,0.335356,0.020549
3,36.742250,-76.570730,60,3,3.0,36.742250,-76.570730,60.0,3.0,3.0,...,0.030374,0.030625,0.000871,0.001081,0.004264,0.001583,0.000938,0.066629,0.566361,0.046726
4,36.742710,-76.585270,60,4,4.0,36.742710,-76.585270,60.0,4.0,4.0,...,0.115285,0.077527,0.016062,0.006030,0.008989,0.001771,0.002487,0.190740,0.335356,0.020549
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2539,42.686200,-74.918400,103,2539,2535.0,42.686200,-74.918400,103.0,2539.0,2539.0,...,0.049736,0.119358,0.002658,0.004245,0.000437,0.001878,0.251831,0.075340,0.096111,0.009733
2540,42.690481,-74.929575,87,2540,2536.0,42.690481,-74.929575,87.0,2540.0,2540.0,...,0.048387,0.105101,0.002246,0.003939,0.000626,0.001495,0.250837,0.080150,0.091555,0.009629
2541,42.710900,-76.151600,4,2541,2537.0,42.710900,-76.151600,4.0,2541.0,2541.0,...,0.032380,0.034924,0.004998,0.001516,0.009317,0.001875,0.151067,0.221712,0.052151,0.006693
2542,42.776900,-76.153400,12,2542,2538.0,42.776900,-76.153400,12.0,2542.0,2542.0,...,0.032380,0.034924,0.004998,0.001516,0.009317,0.001875,0.151067,0.221712,0.052151,0.006693


In [4]:
#remove nulls
true_locations_streamflow = true_locations_streamflow.dropna(subset=['FID'])
true_locations_streamflow

,Latitude,Longitude,numEntries,ID,FID,Field1,Field2,Field3,FullID,FullID_Str,...,PctVALUE_42,PctVALUE_43,PctVALUE_45,PctVALUE_46,PctVALUE_52,PctVALUE_71,PctVALUE_81,PctVALUE_82,PctVALUE_90,PctVALUE_95
0,36.740040,-76.584300,60,0,0.0,36.740040,-76.584300,60.0,0.0,0.0,...,0.115285,0.077527,0.016062,0.006030,0.008989,0.001771,0.002487,0.190740,0.335356,0.020549
1,36.740257,-76.579794,504,1,1.0,36.740257,-76.579794,504.0,1.0,1.0,...,0.115285,0.077527,0.016062,0.006030,0.008989,0.001771,0.002487,0.190740,0.335356,0.020549
2,36.740970,-76.577900,70,2,2.0,36.740970,-76.577900,70.0,2.0,2.0,...,0.115285,0.077527,0.016062,0.006030,0.008989,0.001771,0.002487,0.190740,0.335356,0.020549
3,36.742250,-76.570730,60,3,3.0,36.742250,-76.570730,60.0,3.0,3.0,...,0.030374,0.030625,0.000871,0.001081,0.004264,0.001583,0.000938,0.066629,0.566361,0.046726
4,36.742710,-76.585270,60,4,4.0,36.742710,-76.585270,60.0,4.0,4.0,...,0.115285,0.077527,0.016062,0.006030,0.008989,0.001771,0.002487,0.190740,0.335356,0.020549
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2539,42.686200,-74.918400,103,2539,2535.0,42.686200,-74.918400,103.0,2539.0,2539.0,...,0.049736,0.119358,0.002658,0.004245,0.000437,0.001878,0.251831,0.075340,0.096111,0.009733
2540,42.690481,-74.929575,87,2540,2536.0,42.690481,-74.929575,87.0,2540.0,2540.0,...,0.048387,0.105101,0.002246,0.003939,0.000626,0.001495,0.250837,0.080150,0.091555,0.009629
2541,42.710900,-76.151600,4,2541,2537.0,42.710900,-76.151600,4.0,2541.0,2541.0,...,0.032380,0.034924,0.004998,0.001516,0.009317,0.001875,0.151067,0.221712,0.052151,0.006693
2542,42.776900,-76.153400,12,2542,2538.0,42.776900,-76.153400,12.0,2542.0,2542.0,...,0.032380,0.034924,0.004998,0.001516,0.009317,0.001875,0.151067,0.221712,0.052151,0.006693


In [5]:
#all done, save file
true_locations_streamflow.to_csv('TrueLocationsStreamFlow.csv')

'TrueLocationsStreamFlow.csv' provides the percent landcover of all HUC12 upstream from each CBP monitoring station. This could be of use for other data exploration or machine learning models.

In [7]:
#add HUC landcover values to pivoted data table

#measurements data
file_location = 'Water_FINAL_pivoted_df_CBP.csv'
datadf = pd.read_csv(file_location)

#get HUCS
datadf_HUC = pd.read_csv('HUC_plain.csv')
datadf_HUC = datadf_HUC[['HUC12', 'HUC10', 'HUC8']]

#names of columns
percents = ['V_0_PERCENT','V_11_PERCENT','V_21_PERCENT','V_22_PERCENT','V_23_PERCENT','V_24_PERCENT',
'V_31_PERCENT','V_41_PERCENT','V_42_PERCENT','V_43_PERCENT','V_45_PERCENT','V_46_PERCENT',
'V_52_PERCENT','V_71_PERCENT','V_81_PERCENT','V_82_PERCENT','V_90_PERCENT','V_95_PERCENT']
#get HUC8
HUC_8 = pd.read_csv('HUC8_wPercents.csv')
HUC_8 = HUC_8[['HUC8'] + percents]

#get HUC10
HUC_10 = pd.read_csv('HUC10_wPercents.csv')
HUC_10 = HUC_10[['HUC10'] + percents]

#get HUC12
HUC_12 = pd.read_csv('HUC12_wPercents.csv')
HUC_12 = HUC_12[['HUC12'] + percents]

datadf_HUC.head()

,HUC12,HUC10,HUC8
0,20402070401,204020704,2040207
1,20402070402,204020704,2040207
2,20402070403,204020704,2040207
3,20402070501,204020705,2040207
4,20402070502,204020705,2040207


In [8]:
HUC_12

,HUC12,V_0_PERCENT,V_11_PERCENT,V_21_PERCENT,V_22_PERCENT,V_23_PERCENT,V_24_PERCENT,V_31_PERCENT,V_41_PERCENT,V_42_PERCENT,V_43_PERCENT,V_45_PERCENT,V_46_PERCENT,V_52_PERCENT,V_71_PERCENT,V_81_PERCENT,V_82_PERCENT,V_90_PERCENT,V_95_PERCENT
0,20801010000,0.0,0.999469,0.000386,0.000073,0.000054,0.000011,0.000007,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,20802080204,0.0,0.068497,0.196864,0.346835,0.213576,0.094385,0.001253,0.001639,0.008026,0.007914,0.000009,0.000069,0.000652,0.004266,0.002858,0.002627,0.039210,0.011322
2,20801080101,0.0,0.414791,0.118128,0.061479,0.027174,0.005664,0.001089,0.008063,0.087645,0.048614,0.001132,0.000792,0.007421,0.000857,0.001304,0.014719,0.110874,0.090254
3,20801080104,0.0,0.492227,0.085325,0.100358,0.027869,0.006063,0.008871,0.001669,0.020353,0.002412,0.000033,0.000000,0.001883,0.000430,0.003172,0.010622,0.074290,0.164423
4,20802080102,0.0,0.032946,0.056522,0.024862,0.012996,0.002495,0.000843,0.034355,0.166723,0.095762,0.019438,0.011962,0.010093,0.001678,0.002425,0.330673,0.193280,0.002947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2075,20600020403,0.0,0.003908,0.047183,0.009932,0.002840,0.001429,0.000034,0.079528,0.002806,0.019211,0.000241,0.000241,0.000585,0.000034,0.007092,0.609746,0.214707,0.000482
2076,20600020404,0.0,0.004160,0.041733,0.015568,0.002914,0.001029,0.000072,0.078973,0.002624,0.017018,0.000000,0.000174,0.000594,0.000188,0.001725,0.652437,0.179674,0.001116
2077,20600020405,0.0,0.012285,0.031210,0.007373,0.001963,0.001143,0.000166,0.044677,0.000078,0.003887,0.000049,0.000215,0.000059,0.000107,0.006924,0.824193,0.055595,0.010078
2078,20700070303,0.0,0.059433,0.132233,0.008871,0.000333,0.000000,0.000024,0.340357,0.003781,0.255714,0.000048,0.001094,0.000119,0.000119,0.152639,0.043190,0.001950,0.000095


In [9]:
#combine tables

datadf = pd.merge(datadf, datadf_HUC, how = 'left', left_on = 'HUC12_', right_on = 'HUC12')
datadf = pd.merge(datadf, HUC_12, how = 'left', left_on = 'HUC12', right_on = 'HUC12', suffixes=('','_12'))
datadf = pd.merge(datadf, HUC_10, how = 'left', left_on = 'HUC10', right_on = 'HUC10',suffixes=('','_10'))
datadf = pd.merge(datadf, HUC_8, how = 'left', left_on = 'HUC8', right_on = 'HUC8', suffixes=('','_8'))
datadf

,Station,Latitude,Longitude,HUC12_,HUCNAME_,FIPS_,STATE_,COUNTY_,DateTime,ACTIVE CHLOROPHYLL-A,...,V_42_PERCENT_8,V_43_PERCENT_8,V_45_PERCENT_8,V_46_PERCENT_8,V_52_PERCENT_8,V_71_PERCENT_8,V_81_PERCENT_8,V_82_PERCENT_8,V_90_PERCENT_8,V_95_PERCENT_8
0,1491000,38.99722,-75.78611,20600050203,Gravelly Branch-Choptank River,24011,MD,Caroline County,10/12/2011 9:00,NaN,...,0.020225,0.034919,0.001003,0.000254,0.003295,0.000502,0.003417,0.373973,0.151438,0.041389
1,1491000,38.99722,-75.78611,20600050203,Gravelly Branch-Choptank River,24011,MD,Caroline County,11/7/2011 11:00,NaN,...,0.020225,0.034919,0.001003,0.000254,0.003295,0.000502,0.003417,0.373973,0.151438,0.041389
2,1491000,38.99722,-75.78611,20600050203,Gravelly Branch-Choptank River,24011,MD,Caroline County,12/1/2011 10:30,NaN,...,0.020225,0.034919,0.001003,0.000254,0.003295,0.000502,0.003417,0.373973,0.151438,0.041389
3,1491000,38.99722,-75.78611,20600050203,Gravelly Branch-Choptank River,24011,MD,Caroline County,12/8/2011 11:30,NaN,...,0.020225,0.034919,0.001003,0.000254,0.003295,0.000502,0.003417,0.373973,0.151438,0.041389
4,1491000,38.99722,-75.78611,20600050203,Gravelly Branch-Choptank River,24011,MD,Caroline County,1/9/2012 10:30,NaN,...,0.020225,0.034919,0.001003,0.000254,0.003295,0.000502,0.003417,0.373973,0.151438,0.041389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85981,ZDM0002,38.93648,-76.50738,20600040302,Beards Creek-South River,24003,MD,Anne Arundel County,8/19/2008 8:15,43.361,...,0.012792,0.112074,0.001027,0.001790,0.006637,0.001297,0.023211,0.043898,0.082149,0.010149
85982,ZDM0002,38.93648,-76.50738,20600040302,Beards Creek-South River,24003,MD,Anne Arundel County,9/3/2008 8:45,2.492,...,0.012792,0.112074,0.001027,0.001790,0.006637,0.001297,0.023211,0.043898,0.082149,0.010149
85983,ZDM0002,38.93648,-76.50738,20600040302,Beards Creek-South River,24003,MD,Anne Arundel County,9/17/2008 10:45,20.434,...,0.012792,0.112074,0.001027,0.001790,0.006637,0.001297,0.023211,0.043898,0.082149,0.010149
85984,ZDM0002,38.93648,-76.50738,20600040302,Beards Creek-South River,24003,MD,Anne Arundel County,10/1/2008 10:00,36.383,...,0.012792,0.112074,0.001027,0.001790,0.006637,0.001297,0.023211,0.043898,0.082149,0.010149


In [10]:
a  = datadf.columns
for i in range(0,len(a)):
    print(a[i])

Station
Latitude
Longitude
HUC12_
HUCNAME_
FIPS_
STATE_
COUNTY_
DateTime
ACTIVE CHLOROPHYLL-A
DISSOLVED OXYGEN IN MG/L
AMMONIUM NITROGEN AS N (FILTERED SAMPLE)
NITRATE NITROGEN AS N (FILTERED SAMPLE)
PH CORRECTED FOR TEMPERATURE (25 DEG C)
ORTHOPHOSPHATE PHOSPHORUS AS P (FILTERED SAMPLE)
SALINITY UNITS IN PPT AND EQUAL TO PRACTICAL SALNITY UNITS (PSU)
SECCHI DEPTH
TOTAL ALKALINITY AS CACO3
TOTAL DISSOLVED SOLIDS; GRAVIMETRIC; DRIED AT 180 C
TOTAL KJELDAHL NITROGEN (WHOLE SAMPLE)
TOTAL NITROGEN
TOTAL PHOSPHORUS
TOTAL SUSPENDED SOLIDS
TURBIDITY; NEPHELOMETRIC METHOD
WATER TEMPERATURE DEG
HUC12
HUC10
HUC8
V_0_PERCENT
V_11_PERCENT
V_21_PERCENT
V_22_PERCENT
V_23_PERCENT
V_24_PERCENT
V_31_PERCENT
V_41_PERCENT
V_42_PERCENT
V_43_PERCENT
V_45_PERCENT
V_46_PERCENT
V_52_PERCENT
V_71_PERCENT
V_81_PERCENT
V_82_PERCENT
V_90_PERCENT
V_95_PERCENT
V_0_PERCENT_10
V_11_PERCENT_10
V_21_PERCENT_10
V_22_PERCENT_10
V_23_PERCENT_10
V_24_PERCENT_10
V_31_PERCENT_10
V_41_PERCENT_10
V_42_PERCENT_10
V_43_PERCENT_1

In [11]:
#check for nulls
print(datadf.isnull().sum())

#remove 196 entries
datadf = datadf.dropna(subset=['V_95_PERCENT_8'])
print(datadf.isnull().sum())


Station             0
Latitude            0
Longitude           0
HUC12_              0
HUCNAME_            0
                 ... 
V_71_PERCENT_8    196
V_81_PERCENT_8    196
V_82_PERCENT_8    196
V_90_PERCENT_8    196
V_95_PERCENT_8    196
Length: 82, dtype: int64
Station           0
Latitude          0
Longitude         0
HUC12_            0
HUCNAME_          0
                 ..
V_71_PERCENT_8    0
V_81_PERCENT_8    0
V_82_PERCENT_8    0
V_90_PERCENT_8    0
V_95_PERCENT_8    0
Length: 82, dtype: int64


In [12]:
#save file
datadf.to_csv('WateQuality_wHUC_Percents.csv')